In [74]:
import pandas as pd, numpy as np
import random

In [75]:
# Collect response
# Parameters
address_response = "https://docs.google.com/spreadsheets/d/1UD-iAwpCvhKwEtQYc0arrel_DQbyOt5Qfcz9GDELHZM/edit?resourcekey#gid=1397895293"
name_sheet = "FormResponses1"
l_member = ['藤川 慎也', '池亀 天平', '越山 太輔', '熊倉 陽介', '星野 瑞生', '水谷 真志', '清田 正紘', '市川 貴一', '鈴木 魁士']
l_duty = ['5/2(木)当直', '5/3(金)日直', '5/3(金)当直', '5/4(土)日直', '5/4(土)当直', '5/5(日)日直', '5/5(日)当直', '5/6(月)日直', '5/6(月)当直']
l_rank = ['第1希望', '第2希望', '第3希望', '第4希望', '第5希望', '第6希望', '第7希望', '第8希望', '第9希望']

# Read G sheet
sheet_id = address_response.split('/')[5]
d_preference_src = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={name_sheet}")

# Convert
# col: member, index: rank of preference, value: index of duty
d_preference = pd.DataFrame(columns = [i for i in range(len(l_member))], index = [i for i in range(len(l_duty))])
for i_member, name_member in enumerate(l_member):
    if name_member in d_preference_src['お名前（敬称略）'].tolist():
        s_preference_member = d_preference_src.loc[d_preference_src['お名前（敬称略）'] == name_member, [col.startswith('日当直希望') for col in d_preference_src.columns]]
        for i_duty, name_duty in enumerate(l_duty):
            rank_src = s_preference_member.loc[:, [name_duty in i_pref for i_pref in s_preference_member.columns.tolist()]].values.tolist()[0][0]
            for i_rank, name_rank in enumerate(l_rank):
                if rank_src == name_rank:
                    d_preference.loc[i_rank, i_member] = i_duty

In [26]:
# Test with dummy preference data

seed_rand = 0
random.seed(seed_rand)
n_member = 9

# col: member, index: rank of preference, value: index of duty
d_preference = pd.DataFrame(columns = [i for i in range(n_member)], index = [i for i in range(n_member)])

for i in range(n_member):
    l_shift = [i for i in range(n_member)]
    random.shuffle(l_shift)
    d_preference.loc[:, i] = l_shift

In [76]:
# Apply algorithm
seed_rand = 0
random.seed(seed_rand)

d_match = pd.DataFrame(index = [i for i in range(len(l_duty))], columns = ['member', 'rank']) # index: shift

for round in range(len(l_rank)):
    l_member_free = [i for i in range(len(l_member)) if i not in d_match['member'].tolist()]
    s_round = d_preference.loc[round, :].copy()
    for shift in range(len(l_duty)):
        if np.isnan(d_match.loc[shift, 'member']):
            l_applicant = s_round.loc[s_round == shift].index.tolist() # list of member who applied to the shift in the round(rank)
            l_applicant = [i for i in l_applicant if i in l_member_free]
            if len(l_applicant) > 0:
                d_match.loc[shift, 'member'] = random.sample(l_applicant, 1)[0]
                d_match.loc[shift, 'rank'] = round

In [78]:
# Convert result
d_match['duty'] = l_duty
for i_member, name_member in enumerate(l_member):
    d_match.loc[d_match['member'] == i_member, 'member'] = name_member

d_match = d_match[['duty', 'member', 'rank']]

In [79]:
d_match

,duty,member,rank
0,5/2(木)当直,鈴木 魁士,0
1,5/3(金)日直,星野 瑞生,0
2,5/3(金)当直,藤川 慎也,0
3,5/4(土)日直,池亀 天平,1
4,5/4(土)当直,越山 太輔,3
5,5/5(日)日直,熊倉 陽介,6
6,5/5(日)当直,清田 正紘,1
7,5/6(月)日直,市川 貴一,0
8,5/6(月)当直,水谷 真志,0
